## xT de una temporada de una competicion

In [1]:
import pandas as pd
from socceraction.data.statsbomb import StatsBombLoader
import socceraction.spadl as spadl
import socceraction.xthreat as xthreat

# 1. Load a set of actions to train the model on
SBL = StatsBombLoader()
df_games = SBL.games(competition_id=11, season_id=90)
dataset = [
    {
        **game,
        'actions': spadl.statsbomb.convert_to_actions(
            events=SBL.events(game['game_id']),
            home_team_id=game['home_team_id']
        )
    }
    for game in df_games.to_dict(orient='records')
]

# 2. Convert direction of play + add names
df_actions_ltr = pd.concat([
  spadl.play_left_to_right(game['actions'], game['home_team_id'])
  for game in dataset
])
df_actions_ltr = spadl.add_names(df_actions_ltr)

# 3. Train xT model with 16 x 12 grid
xTModel = xthreat.ExpectedThreat(l=16, w=12)
xTModel.fit(df_actions_ltr)

# 4. Rate ball-progressing actions
# xT should only be used to value actions that move the ball
# and that keep the current team in possession of the ball
df_mov_actions = xthreat.get_successful_move_actions(df_actions_ltr)
df_mov_actions["xT_value"] = xTModel.rate(df_mov_actions)

C:\Users\Arnau Climent\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
C:\Users\Arnau Climent\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
C:\Users\Arnau Climent\AppData\Local\Programs\Python\Python311\Lib\site-packages\socceraction\data\statsbomb\loader.py:337: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  eventsdf["under_pressure"] = eventsdf["under_pressure"].fillna(False).astype(bool)
C:\Users\Arnau Climent\AppData\Local\Programs\Python\Python311\Lib\site-packages\socceraction\data\statsbomb\loader.py:338: FutureWarni

# iterations:  62


In [2]:
df_mov_actions

,game_id,original_event_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,type_id,result_id,bodypart_id,action_id,type_name,result_name,bodypart_name,xT_value
0,3773386,208f072d-cffa-4575-bd73-e9e9384c8fe1,1,0.689,217,5487.0,53.33125,33.9575,43.70625,29.4525,0,1,4,0,pass,success,foot_left,-0.002441
1,3773386,e1f11e14-fb24-4c82-868f-507385122b9c,1,1.653,217,5203.0,43.18125,29.8775,41.16875,42.8825,0,1,5,1,pass,success,foot_right,-0.000163
2,3773386,e5a0193d-1989-4c55-89e2-2f84c0a9ee7b,1,2.652,217,8118.0,41.16875,42.8825,42.04375,42.8825,21,1,0,2,dribble,success,foot,0.000000
3,3773386,4d801bc4-907a-4474-9620-7c6ca2610ea7,1,5.344,217,8118.0,42.04375,42.8825,31.01875,25.0325,0,1,5,3,pass,success,foot_right,-0.002904
4,3773386,9c9ef7ce-4d9d-4c9c-b5db-9c315e2947ed,1,6.608,217,5213.0,31.01875,25.0325,38.71875,25.3725,21,1,0,4,dribble,success,foot,0.001162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231,3773477,f35e9097-adfe-432c-9d3f-c5fe3acee513,2,2968.515,218,23522.0,24.71875,8.7975,24.71875,8.7975,21,1,0,2231,dribble,success,foot,0.000000
2232,3773477,597c1e3b-6681-43d5-b92d-be9f2e5595c4,2,2969.892,218,23522.0,24.71875,8.7975,36.70625,3.4425,0,1,5,2232,pass,success,foot_right,0.001805
2233,3773477,0af48e58-6d5d-4087-a8d0-a7b5d1931edb,2,2970.866,218,6851.0,36.70625,3.4425,36.70625,3.4425,21,1,0,2233,dribble,success,foot,0.000000
2236,3773477,25cea3c4-6bff-44ab-915b-5773638e3c64,2,3006.018,217,20055.0,6.51875,35.0625,7.74375,31.1525,21,1,0,2236,dribble,success,foot,0.000655


In [3]:
# Primero, carga los partidos de la competición y temporada especificados
df_games = SBL.games(competition_id=11, season_id=90)
print('\nhay:',df_games.shape[0],' partidos\n\n')

# Luego, extrae los jugadores de cada partido en esa competición
df_players = []
for game in df_games['game_id']:
    players = SBL.players(game_id=game)
    df_players.append(players)

# Concatenar todos los jugadores en un solo DataFrame
df_players_all = pd.concat(df_players, ignore_index=True)

# Ver los jugadores
df_players_all



hay: 35  partidos




C:\Users\Arnau Climent\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
C:\Users\Arnau Climent\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
C:\Users\Arnau Climent\AppData\Local\Programs\Python\Python311\Lib\site-packages\socceraction\data\statsbomb\loader.py:337: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  eventsdf["under_pressure"] = eventsdf["under_pressure"].fillna(False).astype(bool)
C:\Users\Arnau Climent\AppData\Local\Programs\Python\Python311\Lib\site-packages\socceraction\data\statsbomb\loader.py:338: FutureWarni

,game_id,team_id,player_id,player_name,nickname,jersey_number,is_starter,starting_position_id,starting_position_name,minutes_played
0,3773386,206,3093,Florian Grégoire Claude Lejeune,None,22,True,5,Left Center Back,96
1,3773386,206,3265,José Luis Sanmartín Mato,Joselu,9,False,0,Substitute,38
2,3773386,206,6612,Rubén Duarte Sánchez,Rubén Duarte,3,True,6,Left Back,96
3,3773386,206,6615,Víctor Laguardia Cisneros,Víctor Laguardia,5,True,3,Right Center Back,96
4,3773386,206,6618,Martín Aguirregabiria Padilla,Martín Aguirregabiria,21,False,0,Substitute,56
...,...,...,...,...,...,...,...,...,...,...
1065,3773477,218,7910,Lorenzo Jesús Morón García,Loren Morón,16,False,0,Substitute,25
1066,3773477,218,11538,Alejandro Moreno Lopera,Moreno Lopera,15,True,6,Left Back,98
1067,3773477,218,23522,Emerson Aparecido Leite de Souza Junior,Emerson Royal,22,True,2,Right Back,98
1068,3773477,218,26404,Guido Rodríguez,None,21,True,15,Left Center Midfield,98


In [4]:
#agrupar por jugador para entender que jugadores fueron los de más xT.
df_mov_actions_2 = df_mov_actions.groupby('player_id').sum()['xT_value'].sort_values(ascending=False)
df_mov_actions_2

player_id
5503.0     13.378817
5211.0      9.969040
5477.0      4.452565
6826.0      4.127615
8118.0      4.016073
             ...    
23947.0    -0.115174
6721.0     -0.131231
20033.0    -0.134711
23944.0    -0.142531
24681.0    -0.161118
Name: xT_value, Length: 382, dtype: float64

In [5]:
df_players_all_clean = df_players_all.drop_duplicates(subset=['player_id'])
df_players_all_clean

,game_id,team_id,player_id,player_name,nickname,jersey_number,is_starter,starting_position_id,starting_position_name,minutes_played
0,3773386,206,3093,Florian Grégoire Claude Lejeune,None,22,True,5,Left Center Back,96
1,3773386,206,3265,José Luis Sanmartín Mato,Joselu,9,False,0,Substitute,38
2,3773386,206,6612,Rubén Duarte Sánchez,Rubén Duarte,3,True,6,Left Back,96
3,3773386,206,6615,Víctor Laguardia Cisneros,Víctor Laguardia,5,True,3,Right Center Back,96
4,3773386,206,6618,Martín Aguirregabiria Padilla,Martín Aguirregabiria,21,False,0,Substitute,56
...,...,...,...,...,...,...,...,...,...,...
1055,3773477,218,4691,Claudio Andrés Bravo Muñoz,Claudio Bravo,25,True,1,Goalkeeper,98
1056,3773477,218,5214,William Silva de Carvalho,William Carvalho,14,True,13,Right Center Midfield,98
1058,3773477,218,6651,Joaquín Sánchez Rodríguez,Joaquín,17,True,17,Right Wing,64
1063,3773477,218,6880,Arnaldo Antonio Sanabria Ayala,Antonio Sanabria,19,True,23,Center Forward,73


In [6]:
# Unir con df_players para agregar el nombre del jugador
df_mov_actions_2 = df_mov_actions_2.reset_index()  # Restablecer el índice para tener 'player_id' como columna
df_merged = df_mov_actions_2.merge(df_players_all_clean[['player_id', 'player_name']], on='player_id', how='left')

# Mostrar el DataFrame con los nombres de los jugadores
df_merged

,player_id,xT_value,player_name
0,5503.0,13.378817,Lionel Andrés Messi Cuccittini
1,5211.0,9.969040,Jordi Alba Ramos
2,5477.0,4.452565,Ousmane Dembélé
3,6826.0,4.127615,Clément Lenglet
4,8118.0,4.016073,Frenkie de Jong
...,...,...,...
377,23947.0,-0.115174,Roberto Torres Morales
378,6721.0,-0.131231,Willian José da Silva
379,20033.0,-0.134711,Luuk de Jong
380,23944.0,-0.142531,Oier Sanjurjo Mate
